### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [1]:
! pip install -r phi-chat-requirements.txt --quiet

### Provision the sample

This sample uses [`azd`](https://learn.microsoft.com/azure/developer/azure-developer-cli/), a bicep template, and a custom post-provision hook to provision the sample. The sample uses role-based authentication using your identity for authentication. API keys are used to authenticate to the Phi 3 endpoint.

1. Open a PowerShell command prompt in the phi-chat folder.

1. Run `azd config set defaults.subscription <yourSubscriptionID>` to set the subscription if you have multiple Azure subscriptions.
1. Run `azd env new <your-environment-name>` to create a new `azd` environment. This environment will contain a set of environment variables used to provision the sample.
1. If you want to use an existing resource, set the corresponding `azd` environment variables before deployment:
   1. Existing Search resource:
      1. `azd env set AZURE_SEARCH_SERVICE <your-search-service-name>`
      1. `azd env set AZURE_SEARCH_SERVICE_LOCATION <your-search-service-location>`
      1. `azd env set AZURE_SEARCH_SERVICE_RESOURCE_GROUP <your-search-service-resource-group>`
      1. `azd env set AZURE_SEARCH_SERVICE_SKU <your-search-service-sku>`
      1. `azd env set AZURE_SEARCH_SERVICE_SEMANTIC_RANKER <your-semantic-ranker-sku>`
   1. Existing Azure OpenAI account:
      1. `azd env set AZURE_OPENAI_ACCOUNT <your-openai-account-name>`
      1. `azd env set AZURE_OPENAI_LOCATION <your-openai-account-location>`
      1. `azd env set AZURE_OPENAI_RESOURCE_GROUP <your-openai-resource-group>`
   1. By default, `text-embedding-3-large` with 3072 dimensions is used to embed the sample documents. You can change this by setting the following environment variables:
      1. `azd env set AZURE_OPENAI_EMB_MODEL_NAME <embedding-model-name>`
      1. `azd env set AZURE_OPENAI_EMB_MODEL_VERSION <embedding-model-version>`
      1. `azd env set AZURE_OPENAI_EMB_MODEL_DIMENSIONS <embedding-model-dimensions>`
      1. `azd env set AZURE_OPENAI_EMB_DEPLOYMENT <embedding-model-deployment-name>`
      1. `azd env set AZURE_OPENAI_EMB_DEPLOYMENT_CAPACITY <embedding-model-deployment-capacity>`
   1. By default, gpt-35-turbo and gpt-4 are deployed alongside Phi 3 for comparison. You can disable and configure these deployments using the following environment variables:
      1. `azd env set ENABLE_GPT35TURBO false`
      1. `azd env set ENABLE_GPT4 false`
   1. You can use an existing AI Hub, AI Project, Online Endpoint, or deployment by setting the following environment variables:
      1. `azd env set AZUREAI_HUB_NAME <ai-hub-name>`
      1. `azd env set AZUREAI_PROJECT_NAME <ai-project-name>`
      1. `azd env set AZUREAI_ONLINE_ENDPOINT_NAME <ai-online-endpoint-name>`
      1. `azd env set AZUREAI_DEPLOYMENT_NAME <ai-deployment-name>`
   1. You can change which model is deployed by pointing to a different model on the [AzureML Registry](https://learn.microsoft.com/azure/machine-learning/how-to-manage-models) by setting the following environment variable:
      1. `azd env set AZURE_DEPLOYMENT_MODEL <azureml://registries/registry/models/model/versions/version`
   1. You can change the compute type and deployment capacity by setting the following environment variables:
      1. `azd env set AZURE_DEPLOYMENT_INSTANCE_TYPE <compute-instance-type>`
      1. `azd env set AZURE_DEPLOYMENT_CAPACITY <compute-capacity>`
1. Run `azd provision`.
   1. Enter a development environment name.
   1. Enter a region for the deployment. Be sure to choose a region you have capacity for a Phi 3 deployment in.

This step may take up to a half an hour to complete. Provisioning compute for Phi 3 may take a long time.

### Retrieve environment variables after provisioning

The included `azd` bicep template saves all required environment variables for the notebook automatically.

In [2]:
# Load all environment variables from the azd deployment
import subprocess
from io import StringIO
from dotenv import load_dotenv
import os
result = subprocess.run("azd env get-values", stdout=subprocess.PIPE, cwd=os.getcwd())
load_dotenv(override=True, stream=StringIO(result.stdout.decode("utf-8")))

True

## Validate the indexer has completed successfully

An indexer runs in the background to chunk and vectorize all the sample documents. Validate that it has completed without any errors before trying to search the sample index. If there are any errors, they may be due to temporary throttling from the Azure OpenAI embeddings. Try re-running the indexer to resolve this issue.

In [3]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.identity import DefaultAzureCredential

search_indexer_client = SearchIndexerClient(endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"), credential=DefaultAzureCredential())
status = search_indexer_client.get_indexer_status(name=os.getenv("AZURE_SEARCH_INDEXER"))
print(f"Status: {status.last_result.status}")

Status: success


## Get Phi 3 scoring URI and key

Connect to the Azure AI Project and fetch the deployed Phi scoring uri and key for access

In [4]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

workspace_ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id=os.getenv("AZURE_SUBSCRIPTION_ID"),
    resource_group_name=os.getenv("AZURE_RESOURCE_GROUP"),
    workspace_name=os.getenv("AZUREAI_PROJECT_NAME")
)

phi3_url = workspace_ml_client.online_endpoints.get(name=os.getenv("AZUREAI_ONLINE_ENDPOINT_NAME")).scoring_uri
phi3_key = workspace_ml_client.online_endpoints.get_keys(name=os.getenv("AZUREAI_ONLINE_ENDPOINT_NAME")).primary_key

# Setup search parameters for grounding the conversation on your documents

Change the various types of searches to change how chunks are fetched from the index.

* `k` - Determines how many documents are fetched by either the vector search step and the text portion of the hybrid search step. This parameter is used to improve search result quality.
* `search_type` - `text` for plain text search, `vector` for plain semantic similiarity search using an OpenAI embedding model, or `hybrid` to combine both text and semantic similarity search using Reciprocal Rank Fusion (RRF).
* `use_semantic_reranker` - Whether or not to rerank the top results using the semantic reranker model for improved accuracy.
* `sources_to_include` - How many documents to send to the Language Model for RAG.

In [5]:
k=50
search_type="hybrid"
use_semantic_reranker=True
sources_to_include=5

## Chat using Phi 3

Start the conversation using Phi 3 using a minimal RAG implementation located in [lib/chat.py](./lib/chat.py). To increase the amount of tokens Phi 3 is allowed to respond with, change the `max_new_tokens` parameter in `get_phi3_response` to a larger number. Note that the sample deploys the Phi 3 version with a 4k context limit by default.

In [7]:
import azure.identity.aio
from lib.chat import ChatThread, create_search_client, SearchType

chat_thread = ChatThread()

async with azure.identity.aio.DefaultAzureCredential() as credential, create_search_client(credential) as search_client:
    await chat_thread.append_grounded_message(
        search_client=search_client,
        query="What is included in my Northwind Health Plus plan that is not in standard?",
        search_type=SearchType(search_type),
        use_semantic_reranker=use_semantic_reranker,
        sources_to_include=sources_to_include,
        k=k)
    await chat_thread.get_phi3_response(
        endpoint_scoring_uri=phi3_url,
        endpoint_authorization="Bearer " + phi3_key,
        deployment=os.getenv("AZUREAI_DEPLOYMENT_NAME"),
        max_new_tokens=200)

print(chat_thread.get_last_message()["content"])


Northwind Health Plus includes coverage for mental health and substance abuse treatments, unlike Northwind Standard. 

Northwind Standard does not cover mental health and substance abuse treatments. Northwind Health Plus offers a comprehensive plan that includes emergency services, mental health and substance abuse coverage, and out-of-network services, which are not covered by Northwind Standard. Additionally, Northwind Health Plus provides more prescription drug coverage compared to Northwind Standard.

If you require emergency services, ensure your provider is part of the Northwind Health network to avoid full cost responsibility.

For mental health and substance abuse treatments, verify that your provider is networked with Northwind Health Plus.

Coordinate your benefits with other health care plans if you have additional coverage.

Northwind Health Plus covers emergency services, mental health and substance abuse treatments, and out-of-network


## (Optional) Chat using GPT4 or GPT-35-Turbo

Continue your conversation using OpenAI models

In [8]:
from lib.chat import ChatThread, create_search_client, create_openai_client, SearchType

chat_deployment = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT_NAME") # replace with AZURE_OPENAI_GPT35TURBO_DEPLOYMENT_NAME to use gpt-35-turbo

async with azure.identity.aio.DefaultAzureCredential() as credential, create_search_client(credential) as search_client, create_openai_client(credential) as openai_client:
    await chat_thread.append_grounded_message(
        search_client=search_client,
        query="What is included in my Northwind Health Plus plan that is not in standard?",
        search_type=SearchType(search_type),
        use_semantic_reranker=use_semantic_reranker,
        sources_to_include=sources_to_include,
        k=k)
    await chat_thread.get_openai_response(openai_client=openai_client, model=chat_deployment)

print(chat_thread.get_last_message()["content"])


- Coverage for hospital stays, X-rays, and a broader range of prescription drugs not included in Northwind Standard.
- Access to both in-network and out-of-network emergency services.
- Personal health support programs, including case management, disease management, and wellness programs exclusive to Northwind Health Plus.
- Northwind Health Plus includes services by independent contractors, if medically necessary.
- Offers mental health and substance abuse coverage, which are also included in Northwind Standard.
- Coverage for out-of-network services, unlike Northwind Standard.



## Inspect sources used by the Language Model

Check which sources were used to ground the response

In [9]:
chat_thread.get_last_message_sources()

[{'content': 'includes counselling, psychotherapy, \n\nand other treatments related to mental health and substance abuse.  \n\n\n\nOut-of-Network Services: The Northwind Standard plan does not cover any services that are \n\nprovided by a provider that is not part of the Northwind Health network. This includes \n\ndoctors, hospitals, and other healthcare providers who are not part of the Northwind Health \n\nnetwork.  \n\nTips \n\nWhen selecting a healthcare plan, it is important to be aware of the exclusions in the plan. \n\nHere are some tips to help you understand the exclusions in the Northwind Standard plan:  \n\n1. Understand the types of services that are not covered by the Northwind Standard plan. Be \n\nsure to familiarize yourself with the list of exclusions and make sure that any services you \n\nmight require are covered.  \n\n2. If you require emergency services, be sure to check with your provider to see if they are \n\npart of the Northwind Health network. If they are no

### 